In [8]:
%%sh
pip install delta-spark==2.1.1

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import pyspark
from delta import *
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import json
from pyspark.sql import types
from delta import *
from pyspark.sql.functions import *
from delta.tables import *
from pyspark.sql.functions import *
from datetime import datetime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print(spark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%sh
ls ../../data_set

characters.csv
characters_to_Comics.csv
charcters_stats.csv
comics.csv
delta
marvel_characters_info.csv
marvel_dc_characters.csv
marvel_dc_characters.xlsx
superheroes_power_matrix.csv


#### Carregando dados a partir de um schema json

In [29]:
nam_path='/home/glue_user/workspace/jupyter_workspace/data_set'
json_path='/home/glue_user/workspace/jupyter_workspace/src/bronze_data/schemas'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
nam_file='superheroes_power_matrix'
bol_header=True

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
with open(f"{json_path}/{nam_file}.json",'r') as f:
    data = json.load(f)
schemaFromJson = StructType.fromJson(data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df=spark.read.csv(f"{nam_path}/{nam_file}.csv",header=True,schema=schemaFromJson)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Name: string (nullable = true)
 |-- Agility: boolean (nullable = true)
 |-- Accelerated Healing: boolean (nullable = true)
 |-- Lantern Power Ring: boolean (nullable = true)
 |-- Dimensional Awareness: boolean (nullable = true)
 |-- Cold Resistance: boolean (nullable = true)
 |-- Durability: boolean (nullable = true)
 |-- Stealth: boolean (nullable = true)
 |-- Energy Absorption: boolean (nullable = true)
 |-- Flight: boolean (nullable = true)
 |-- Danger Sense: boolean (nullable = true)
 |-- Underwater breathing: boolean (nullable = true)
 |-- Marksmanship: boolean (nullable = true)
 |-- Weapons Master: boolean (nullable = true)
 |-- Power Augmentation: boolean (nullable = true)
 |-- Animal Attributes: boolean (nullable = true)
 |-- Longevity: boolean (nullable = true)
 |-- Intelligence: boolean (nullable = true)
 |-- Super Strength: boolean (nullable = true)
 |-- Cryokinesis: boolean (nullable = true)
 |-- Telepathy: boolean (nullable = true)
 |-- Energy Armor: boolean (nul

In [34]:
df.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------+-------------------+------------------+---------------------+---------------+----------+-------+-----------------+------+------------+--------------------+------------+--------------+------------------+-----------------+---------+------------+--------------+-----------+---------+------------+-------------+-----------+-------------+---------------+-------+-------------+-------------+---------+--------+-----------+----------+----------------------+-------------------+--------------+----------------------+-----------+-------------+---------------+-----------+------------+-----+------------+-----+------------+------------------+-----------+-------------+--------------------+-----------+----------+---------------+-------+-----------------+--------------------+------------+----------+---------+---------------+--------+---------------+-----------------+------------+---------------+------------+-------+---------------+-----------------+--------------+----------+---------

In [41]:
def load_df_with_schema(nam_file, desc_schema):
    with open(desc_schema,'r') as f:
        data = json.load(f)
    schemaFromJson = StructType.fromJson(data)
    df=spark.read.csv(nam_file,header=True,schema=schemaFromJson)
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
nam_path='/home/glue_user/workspace/jupyter_workspace/data_set'
json_path='/home/glue_user/workspace/jupyter_workspace/src/bronze_data/schemas'
nam_file='comics'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df=load_df_with_schema(f"{nam_path}/{nam_file}.csv",f"{json_path}/{nam_file}.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- comicID: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- issueNumber: integer (nullable = true)
 |-- description: string (nullable = true)

In [45]:
df.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------------------------------------------------------------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|comicID|title                                                                             |issueNumber|description                                                                                                                                                                                                                                                                                                                                                             

#### Inclusao de data atual 

In [59]:
dat_load=datetime.now()
dat_load=dat_load.strftime('%Y%m%d_%H:%m:%S')
df=df.withColumn('dat_load',lit(dat_load))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Salvando arquivo delta

In [63]:
df.write.format("delta").mode("overwrite").save(f"{nam_path}/delta/{nam_file}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Verificar path no s3import boto3

In [65]:
import boto3

s3 = boto3.client('s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
nam_bucket='marvel-bronze-dev'
nam_table='characters'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
response = s3.list_objects_v2(Bucket=nam_bucket, Prefix=nam_table)
if response['KeyCount']>1:
    print('existe prefixo')
else:
    print('nao existe prefixo')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

existe prefixo

#### Update da tabela 

In [77]:
deltaTable = DeltaTable.forPath(spark, f"{nam_path}/delta/{nam_file}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
deltaTable.to_df()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DeltaTable' object has no attribute 'to_df'
Traceback (most recent call last):
AttributeError: 'DeltaTable' object has no attribute 'to_df'



In [78]:
deltaTable.alias("oldData") \
  .merge(\
    df.alias("newData"),\
    "oldData.comicID = newData.comicID") \
  .whenMatchedUpdateAll()\
  .whenNotMatchedInsertAll()\
  .execute()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot resolve dat_load in UPDATE clause given columns newData.comicID, newData.title, newData.issueNumber, newData.description
Traceback (most recent call last):
  File "/home/glue_user/.local/lib/python3.10/site-packages/delta/tables.py", line 924, in execute
    self._jbuilder.execute()
  File "/home/glue_user/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/home/glue_user/spark/python/pyspark/sql/utils.py", line 196, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: cannot resolve dat_load in UPDATE clause given columns newData.comicID, newData.title, newData.issueNumber, newData.description



#### Export schema do dataframe

In [ ]:
print(df.schema.json())